# Path Length Similarity (Method 3)

__Anish Sachdeva (DTU/2K16/MC/13)__

__Natural Language Processing (Dr. Seba Susan)__

In the method we are given 2 synsets (words) and we find the smallest hop path between the given synsets $pathlen(c_1, c_2)$ and we compute the similarity as $-\log{pathlen(c_1, c_2)}$.   

In [1]:
# Importing required packages
import nltk
import pickle
import pprint
from nltk.corpus import wordnet
# nltk.download('wordnet')
import pandas as pd
import numpy as np
from scipy import stats

In [2]:
# We define inifinity
infinity = float('inf')

We now define the similarity metric to compute the $sim_{path}(w_1, w_2)$ given the Hypernym Paths of 2 synsets.

In [4]:
def path_similarity(hypernym_path1: list, hypernym_path2: list) -> float:
    """":returns the shortest path similarity metric between 2 hypernym paths"""
    count = 0
    for index, synset in enumerate(hypernym_path1):
        if len(hypernym_path2) <= index or synset != hypernym_path2[index]:
            break
        count += 1
    return -np.log(len(hypernym_path1) + len(hypernym_path2) - 2 * count)

Let us see the hypernyms of a sample word to get a feel for the Hypernym paths. We can se that the word _dog_ for the noun canne has 2 hypernym paths and one particular may be closer to _cat_ and one may be closer _canine_ or _wolf_. 

In [7]:
dog = wordnet.synset('dog.n.01')
pprint.pprint(dog.hypernym_paths())

[[Synset('entity.n.01'),
  Synset('physical_entity.n.01'),
  Synset('object.n.01'),
  Synset('whole.n.02'),
  Synset('living_thing.n.01'),
  Synset('organism.n.01'),
  Synset('animal.n.01'),
  Synset('chordate.n.01'),
  Synset('vertebrate.n.01'),
  Synset('mammal.n.01'),
  Synset('placental.n.01'),
  Synset('carnivore.n.01'),
  Synset('canine.n.02'),
  Synset('dog.n.01')],
 [Synset('entity.n.01'),
  Synset('physical_entity.n.01'),
  Synset('object.n.01'),
  Synset('whole.n.02'),
  Synset('living_thing.n.01'),
  Synset('organism.n.01'),
  Synset('animal.n.01'),
  Synset('domestic_animal.n.01'),
  Synset('dog.n.01')]]


Let us see the similarity metric between _dog_, _cat_ and _wolf_.

In [8]:
cat = wordnet.synset('cat.n.01')
wolf = wordnet.synset('wolf.n.01')

In [14]:
print(path_similarity(dog.hypernym_paths()[0], wolf.hypernym_paths()[0]))

-0.6931471805599453


In [13]:
print(path_similarity(dog.hypernym_paths()[0], cat.hypernym_paths()[0]))

-1.3862943611198906


In [12]:
print(path_similarity(dog.hypernym_paths(), dog.hypernym_paths()))

inf


c:\python37\lib\site-packages\ipykernel_launcher.py:8: RuntimeWarning: divide by zero encountered in log
  


So, we can see that actually a wolf is closer to a dog than a cat is to a dog and a dog is equivalent to a dog, hence the infite similarity metric. We now define a method that will find the maximum similarty between 2 synsets after comparing all possible hypernyms. 

In [16]:
# Define a method return maximum path similarity score given 2 synsets in wordnet
def max_similarity_path(synset_1, synset_2) -> float:
    """:returns the highest path similarity metric score between 2 synsets"""
    max_similarity = -infinity
    for hypernym_path_1 in synset_1.hypernym_paths():
        for hypernym_path_2 in synset_2.hypernym_paths():
            max_similarity = max(max_similarity, path_similarity(hypernym_path_1, hypernym_path_2))
    return max_similarity

We now define a method that will take 2 words (as strings) and will retiurn us 3 things
1. The synset for the first word
1. The synset of the second word
1. The maximum possible similarity score achieved between these synsets

The algorithm will also place the synsets to maximize possible similaritys score.

In [27]:
def closest_synsets(word_1: str, word_2: str):
    """:returns the closest synsets for 2 given words based on path similarity metric"""
    word_1 = wordnet.synsets(word_1.lower())
    word_2 = wordnet.synsets(word_2.lower())
    max_similarity = -float('inf')
    try:
        synset_1_optimal = word_1[0]
        synset_2_optimal = word_2[0]
    except:
        return None, None, -infinity

    for synset_1 in word_1:
        for synset_2 in word_2:
            similarity = max_similarity_path(synset_1, synset_2)
            if max_similarity < similarity:
                max_similarity = similarity
                synset_1_optimal = synset_1
                synset_2_optimal = synset_2

    return synset_1_optimal, synset_2_optimal, max_similarity

We now test this with some sample words:

In [20]:
word_1 = 'dog'
word_2 = 'cat'
word_1_synset, word_2_synset, similarity = closest_synsets(word_1, word_2)
print(word_1.capitalize() + ' Definition:', word_1_synset.definition())
print(word_2.capitalize() + ' Definition:', word_2_synset.definition())
print('similarity:', similarity)

Dog Definition: a member of the genus Canis (probably descended from the common wolf) that has been domesticated by man since prehistoric times; occurs in many breeds
Cat Definition: feline mammal usually having thick soft fur and no ability to roar: domestic cats; wildcats
similarity: -1.3862943611198906


In [21]:
word_1 = 'dog'
word_2 = 'wolf'
word_1_synset, word_2_synset, similarity = closest_synsets(word_1, word_2)
print(word_1.capitalize() + ' Definition:', word_1_synset.definition())
print(word_2.capitalize() + ' Definition:', word_2_synset.definition())
print('similarity:', similarity)

Dog Definition: a member of the genus Canis (probably descended from the common wolf) that has been domesticated by man since prehistoric times; occurs in many breeds
Wolf Definition: any of various predatory carnivorous canine mammals of North America and Eurasia that usually hunt in packs
similarity: -0.6931471805599453


In [22]:
# here we are using verb chase with dog, and let's see what are the closest synsets for this metric
word_1 = 'dog'
word_2 = 'chase'
word_1_synset, word_2_synset, similarity = closest_synsets(word_1, word_2)
print(word_1.capitalize() + ' Definition:', word_1_synset.definition())
print(word_2.capitalize() + ' Definition:', word_2_synset.definition())
print('similarity:', similarity)

Dog Definition: go after with the intent to catch
Chase Definition: go after with the intent to catch
similarity: inf


c:\python37\lib\site-packages\ipykernel_launcher.py:8: RuntimeWarning: divide by zero encountered in log
  


In the above example we see that dog itself means to chase someone and can also be used as a verb and the function we have created selects that to give us maximum similarity. 

## Comapring Keywords in Resume
We previously divided our resume in 6 parts and selected the top 5 words from each part to create a keyword table and we will be referring to each individual part as a document. Now, let us compare the 6th document (Testing data) with the other 5 documents (Training Data). 

### 1. Loading in the 6 documents:

In [23]:
documents = pickle.load(open('../assets/documents.p', 'rb'))
print('The documents are:')
pprint.pprint(documents)

The documents are:
[['python', 'data', 'structures', 'students', 'com', 'delhi'],
 ['java', 'auckland', 'geometry', 'mathematics', 'theory', 'batch'],
 ['cern', 'applications', 'worked', 'research', 'group', 'core'],
 ['worked', 'also', 'requests', 'participated', 'many', 'teaching'],
 ['structures', 'computer', 'algorithms', 'java', 'university', 'mathematics'],
 ['trinity', 'college', 'london', 'plectrum', 'guitar', 'grade']]


### 2. Finding Similarity Between 6th Document and Other Documents

In [28]:
similarity_mat = np.zeros((len(documents) - 1, len(documents[0])))

for column, keyword in enumerate(documents[len(documents) - 1]):
    for row in range(len(documents) - 1):
        similarity_mat[row][column] = closest_synsets(keyword, documents[row][column])[2]

print('The similarity coefficients are:\n')
similarity = pd.DataFrame(similarity_mat, columns=documents[5])
print(similarity.to_string())

The similarity coefficients are:

    trinity   college    london  plectrum    guitar     grade
0 -1.609438 -1.609438 -2.079442 -2.197225      -inf -2.564949
1 -2.302585 -2.302585 -2.772589 -2.708050 -2.708050 -1.098612
2      -inf -2.079442 -2.079442 -2.397895 -2.397895 -1.609438
3 -1.945910 -1.945910 -2.302585 -2.197225 -2.397895 -1.791759
4 -1.609438 -1.945910 -2.639057 -2.079442 -2.079442 -2.302585


### 3. Saving Similarity Coefficients in a Text File for vieweing Results

In [29]:
results = open('../assets/path_similarity_matrix.txt', 'w')
results.write(similarity.to_string())
results.close()

### 4. Selecting Document with Maximum/Minimum Similarity to the 6th Document

In [30]:
min = similarity_mat.argmin(axis=0)
max = similarity_mat.argmax(axis=0)

# document with least/maximum similarity
document_min_similarity = stats.mode(min).mode[0]
document_max_similarity = stats.mode(max).mode[0]

print('Document with Minimum Similarity to 6th document:', documents[document_min_similarity])

Document with Minimum Similarity to 6th document: ['java', 'auckland', 'geometry', 'mathematics', 'theory', 'batch']


In [31]:
print('Document with Maximum Similarity to 6th document:', documents[document_max_similarity])

Document with Maximum Similarity to 6th document: ['python', 'data', 'structures', 'students', 'com', 'delhi']
